# CNNbasic Assignment#2

# AlexNet 구현

모델 구현 후 summary로 전체 모델 구조 출력과 주석으로 간단한 설명을 달아주시면 됩니다.

프레임워크는 자유이고, 기본 tensforflow와 pytorch tutorial 사이트를 아래에 첨부해 드립니다.

이 외 각 프레임워크 별 summary 등 추가적인 사용 방법은 구글링으로 찾아주세요!-!

- Tensorflow Tutorial: https://www.tensorflow.org/tutorials?hl=ko

- Pytorch Tutorial: https://tutorials.pytorch.kr/

![image-2.png](attachment:image-2.png)

## Tensorflow(keras)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense, Dropout
from tensorflow.keras.models import Sequential


n_classes = 1000 # class 개수
input_shape = (224,224, 3)
    
def AlexNet(input_shape, n_classes):
    model = Sequential()
    
    # Conv2D :: filter : 필터의 개수 / input_shape : 입력데이터의 shape / kernel_size : 필터 사이즈 / strides : 각 width, height의 stride / padding : 패딩 수 (valid : no padding)
    # MaxPooling2D :: pool_size : pooling 사이즈 / strides : 위와 동일
    # Flatten() :: 차원 수를 1로 변환
    # Dense  :: units : 출력 뉴런 수 / activation : 활성화 함수
    # Dropout :: p : 드롭아웃 비율 (0 ~ 1)


    # Layer 1
    model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(11,11), strides=(4,4), padding='valid')) 
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    # Layer 2
    model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    # Layer 3-5
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    # Layer 6
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5)) # 논문에서는 dropout에 대해 언급되지 않았지만 표준적으로 각 fc layer의 마지막에 dropout(0.5)를 추가함
    
    # Layer 7
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    
    # Layer 8
    model.add(Dense(n_classes, activation='softmax'))
    
    return model


#######################################

model = AlexNet(input_shape = (224,224,1), n_classes=1000)
# keras summary
model.summary() # summary code 추가

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 54, 54, 96)        11712     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 26, 26, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 12, 12, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 12, 12, 384)       885120    
                                                                 
 conv2d_23 (Conv2D)          (None, 12, 12, 384)      

## Pytorch

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


class AlexNet(nn.Module):

    def __init__(self, n_classes):
        super(AlexNet, self).__init__()

        # nn.Conv2d :: in_channels : 입력 데이터의 채널 수 / out_channels : 출력 데이터의 채널 수 / kernel_size : 필터의 크기 / stride : 필터가 이동하는 간격 / padding : 입력 데이터 주변에 추가하는 추가 픽셀 값(보통 0)
        # nn.MaxPool2d :: kernel_size : 풀링 window의 크기 / stride : 풀링 window가 이동하는 간격
        # nn.Linear :: in_features : 입력 데이터의 특징 수 / out_features : 출력 데이터의 특징 수
        # nn.Dropout :: p : dropout 비율 (0 ~ 1)




        ############## Add Layer ##############
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.drop1 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(4096, 4096)
        self.drop2 = nn.Dropout(0.5)
        
        self.fc3 = nn.Linear(4096, n_classes)

        #######################################

    def forward(self, x):
        
        ############## Add Layer ##############
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool3(F.relu(self.conv5(x)))
        x = x.view(x.size(0), 256 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        x = self.fc3(x)
    
        #######################################
        
        return x


# pytorch summary
model = AlexNet(n_classes=1000)
summary(model, input_size=(3,224,224)) # summary code 추가

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
         MaxPool2d-2           [-1, 96, 27, 27]               0
            Conv2d-3          [-1, 256, 27, 27]         614,656
         MaxPool2d-4          [-1, 256, 13, 13]               0
            Conv2d-5          [-1, 384, 13, 13]         885,120
            Conv2d-6          [-1, 384, 13, 13]       1,327,488
            Conv2d-7          [-1, 256, 13, 13]         884,992
         MaxPool2d-8            [-1, 256, 6, 6]               0
            Linear-9                 [-1, 4096]      37,752,832
          Dropout-10                 [-1, 4096]               0
           Linear-11                 [-1, 4096]      16,781,312
          Dropout-12                 [-1, 4096]               0
           Linear-13                 [-1, 1000]       4,097,000
Total params: 62,378,344
Trainable para